In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,36643
2,Huelva,Confirmados PDIA 14 días,2251
3,Huelva,Tasa PDIA 14 días,"438,6460627082643"
4,Huelva,Confirmados PDIA 7 días,1171
5,Huelva,Tasa PDIA 7 dias,"228,18948886333965"
6,Huelva,Total Confirmados,36844
7,Huelva,Curados,33102
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  36643.0


/tmp/ipykernel_3656/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10207.0


/tmp/ipykernel_3656/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3656/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3656/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3656/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 386 personas en los últimos 7 días 

Un positivo PCR cada 180 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,36643.0,1171.0,2251.0,513170.0,228.189489,438.646063,291.0
Huelva-Costa,21485.0,773.0,1554.0,289548.0,266.967826,536.698578,203.0
Huelva (capital),10207.0,373.0,801.0,143837.0,259.321315,556.880358,106.0
Condado-Campiña,11597.0,263.0,483.0,156231.0,168.340470,309.157594,45.0
Sierra de Huelva-Andévalo Central,3188.0,134.0,200.0,67391.0,198.839608,296.775534,43.0
Isla Cristina,2508.0,80.0,168.0,21393.0,373.954097,785.303604,21.0
Lepe,2410.0,79.0,147.0,27880.0,283.357245,527.259684,20.0
Punta Umbría,961.0,49.0,88.0,15355.0,319.114295,573.103224,18.0
Aljaraque,1340.0,70.0,130.0,21474.0,325.975598,605.383254,17.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,17.0,4.0,4.0,299.0,1337.792642,1337.792642,0.0
Castaño del Robledo,9.0,2.0,2.0,228.0,877.192982,877.192982,0.0
Beas,269.0,22.0,35.0,4341.0,506.795669,806.265837,6.0
Isla Cristina,2508.0,80.0,168.0,21393.0,373.954097,785.303604,21.0
Palos de la Frontera,833.0,44.0,86.0,11742.0,374.723216,732.413558,10.0
Niebla,314.0,7.0,28.0,4116.0,170.068027,680.272109,1.0
Zarza-Perrunal (La),39.0,5.0,8.0,1253.0,399.042298,638.467678,0.0
Santa Ana la Real,14.0,3.0,3.0,475.0,631.578947,631.578947,2.0
Aljaraque,1340.0,70.0,130.0,21474.0,325.975598,605.383254,17.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Niebla,314.0,7.0,28.0,4116.0,170.068027,680.272109,1.0,0.250000
Jabugo,74.0,2.0,7.0,2260.0,88.495575,309.734513,0.0,0.285714
Alosno,210.0,1.0,3.0,3933.0,25.425884,76.277651,0.0,0.333333
Almonaster la Real,73.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
Cerro de Andévalo (El),168.0,3.0,8.0,2327.0,128.921358,343.790288,1.0,0.375000
Moguer,1554.0,46.0,102.0,21867.0,210.362647,466.456304,8.0,0.450980
Huelva (capital),10207.0,373.0,801.0,143837.0,259.321315,556.880358,106.0,0.465668
Isla Cristina,2508.0,80.0,168.0,21393.0,373.954097,785.303604,21.0,0.476190
Palma del Condado (La),1239.0,13.0,27.0,10801.0,120.359226,249.976854,-1.0,0.481481
